## Project Data Wrangling - We rate dogs

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Gathering Data](#gather)
- [Assessing Data](#assess)
- [Cleaning Data](#clean)
- [Analyzing Data](#analyze)




In [4]:
import tweepy
import pandas as pd
import numpy as np
import requests
import os
# use module dotenv to manage API keys and secrets
%load_ext dotenv
%dotenv

consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('CONSUMER_SECRET')
access_token = os.environ.get('ACCESS_TOKEN')
access_secret = os.environ.get('ACCESS_SECRET')
tsv_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)


<a id='gather'></a>
### Gathering Data
We gather data from 3 several sources:
1. enhanced Twitter archive: a csv file 'twitter-archive-enhanced.csv' (data stored in `df_tweets_raw`)
2. Additional Data via the Twitter API
3. Image Predictions File

Let's start with getting the data from twitter archive file and take a look at a few records:

In [7]:
df_tweets_raw = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_raw.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1860,675489971617296384,NaN,NaN,2015-12-12 01:38:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT until we find this dog. Clearly a cool dog ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675489971...,10,10,None,None,None,None,None
900,758740312047005698,NaN,NaN,2016-07-28 19:06:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Boston. He's worried because his tongue w...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758740312...,12,10,Boston,None,None,None,None
1590,686618349602762752,NaN,NaN,2016-01-11 18:39:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Olivia. She just saw an adult wearing ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/686618349...,11,10,Olivia,None,None,pupper,None


<a id='assess'></a>
### Assessing Data

1.) Advanced Twitter Archive



In [14]:
df_tweets_raw.isnull().sum() / df_tweets_raw.shape[0]

tweet_id                      0.000000
in_reply_to_status_id         0.966893
in_reply_to_user_id           0.966893
timestamp                     0.000000
source                        0.000000
text                          0.000000
retweeted_status_id           0.923175
retweeted_status_user_id      0.923175
retweeted_status_timestamp    0.923175
expanded_urls                 0.025042
rating_numerator              0.000000
rating_denominator            0.000000
name                          0.000000
doggo                         0.000000
floofer                       0.000000
pupper                        0.000000
puppo                         0.000000
dtype: float64

In [15]:
df_tweets_raw['doggo'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
2351    False
2352    False
2353    False
2354    False
2355    False
Name: doggo, Length: 2356, dtype: bool

In [18]:
df_tweets_raw['doggo'].replace('None', np.nan, inplace=True)

In [27]:
df_tweets_raw[df_tweets_raw.doggo.notna()].doggo

array(['doggo'], dtype=object)